In [1]:
import torch
import torch.nn as nn
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)


In [2]:
X = torch.FloatTensor([[0, 0], [0, 1], [1, 0], [1, 1]]).to(device)
Y = torch.FloatTensor([[0], [1], [1], [0]]).to(device)


In [3]:
linear = nn.Linear(2, 1, bias=True)
sigmoid = nn.Sigmoid()
model = nn.Sequential(linear, sigmoid).to(device)


In [4]:
# 비용 함수와 옵티마이저 정의
criterion = torch.nn.BCELoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=1)


In [ ]:
for step in range(10001): 
    optimizer.zero_grad()
    hypothesis = model(X)

    # 비용 함수
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    if step % 100 == 0: # 100번째 에포크마다 비용 출력
        print(step, cost.item())


다층 퍼셉트콘 구현

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)


In [7]:
X = torch.FloatTensor([[0, 0], [0, 1], [1, 0], [1, 1]]).to(device)
Y = torch.FloatTensor([[0], [1], [1], [0]]).to(device)


In [8]:
model = nn.Sequential(
          nn.Linear(2, 10, bias=True), # input_layer = 2, hidden_layer1 = 10
          nn.Sigmoid(),
          nn.Linear(10, 10, bias=True), # hidden_layer1 = 10, hidden_layer2 = 10
          nn.Sigmoid(),
          nn.Linear(10, 1, bias=True), # hidden_layer2 = 10, output_layer = 1
          nn.Sigmoid()
          ).to(device)


In [9]:
criterion = torch.nn.BCELoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=1)  # modified learning rate from 0.1 to 1


In [ ]:
for epoch in range(10001):
    optimizer.zero_grad()
    # forward 연산
    hypothesis = model(X)

    # 비용 함수
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    # 100의 배수에 해당되는 에포크마다 비용을 출력
    if epoch % 100 == 0:
        print(epoch, cost.item())


### 손글씨 분류하기

In [10]:
import torch
import torch.nn as nn
from torch import optim


In [1]:
%matplotlib inline
import matplotlib.pyplot as plt # 시각화를 위한 맷플롯립
from sklearn.datasets import load_digits
digits = load_digits() # 1,979개의 이미지 데이터 로드


In [7]:
#숫자로 이미지 출력
print(digits.images[0])
#해당 이미지 숫자 의미 출력
print(digits.target[0])
#images 배열화
print(digits.data[0])


[[ 0.  0.  5. 13.  9.  1.  0.  0.]
 [ 0.  0. 13. 15. 10. 15.  5.  0.]
 [ 0.  3. 15.  2.  0. 11.  8.  0.]
 [ 0.  4. 12.  0.  0.  8.  8.  0.]
 [ 0.  5.  8.  0.  0.  9.  8.  0.]
 [ 0.  4. 11.  0.  1. 12.  7.  0.]
 [ 0.  2. 14.  5. 10. 12.  0.  0.]
 [ 0.  0.  6. 13. 10.  0.  0.  0.]]
0
[ 0.  0.  5. 13.  9.  1.  0.  0.  0.  0. 13. 15. 10. 15.  5.  0.  0.  3.
 15.  2.  0. 11.  8.  0.  0.  4. 12.  0.  0.  8.  8.  0.  0.  5.  8.  0.
  0.  9.  8.  0.  0.  4. 11.  0.  1. 12.  7.  0.  0.  2. 14.  5. 10. 12.
  0.  0.  0.  0.  6. 13. 10.  0.  0.  0.]


In [8]:
X = digits.data # 이미지. 즉, 특성 행렬 : [8x8] 행렬
Y = digits.target # 각 이미지에 대한 레이블 :0

In [ ]:
# 모델 정의: 순차적인 레이어 구조
model = nn.Sequential(
    nn.Linear(64, 32), # 입력층: 64, 첫 번째 은닉층: 32
    nn.ReLU(),         # 활성화 함수: ReLU
    nn.Linear(32, 16), # 첫 번째 은닉층: 32, 두 번째 은닉층: 16
    nn.ReLU(),         # 활성화 함수: ReLU
    nn.Linear(16, 10)  # 두 번째 은닉층: 16, 출력층: 10 (클래스의 개수)
)
#64>32>16>10 레이어 점점 줄어듬


In [12]:
# 입력 데이터 X와 레이블 Y를 텐서로 변환
X = torch.tensor(X, dtype=torch.float32)
Y = torch.tensor(Y, dtype=torch.int64)


In [13]:
# CrossEntropyLoss는 내부적으로 softmax 함수를 포함하고 있습니다.
loss_fn = nn.CrossEntropyLoss() 
#Adam은 학습률을 내부적으로 적응적으로 조정하는 방식으로 최적화를 수행합니다.
optimizer = optim.Adam(model.parameters())
#에포크의 손실(loss) 값을 저장하기 위한 빈 리스트
losses = []


In [14]:
# 총 100번의 에포크 동안 모델 학습
for epoch in range(100):
  optimizer.zero_grad()      # 옵티마이저의 기울기 초기화
  y_pred = model(X)          # 순전파 연산으로 예측값 계산
  loss = loss_fn(y_pred, Y)  # 손실 함수로 비용 계산
  loss.backward()            # 역전파 연산으로 기울기 계산
  optimizer.step()           # 옵티마이저를 통해 파라미터 업데이트

  # 10번째 에포크마다 현재 에포크와 손실 값 출력
  if epoch % 10 == 0:
    print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, 100, loss.item()
        ))

  # 손실 값을 리스트에 추가하여 추적
  losses.append(loss.item())


Epoch    0/100 Cost: 2.306390
Epoch   10/100 Cost: 1.982296
Epoch   20/100 Cost: 1.649993
Epoch   30/100 Cost: 1.267837
Epoch   40/100 Cost: 0.936797
Epoch   50/100 Cost: 0.681193
Epoch   60/100 Cost: 0.498909
Epoch   70/100 Cost: 0.376802
Epoch   80/100 Cost: 0.294665
Epoch   90/100 Cost: 0.237937


In [15]:
print(losses)

[2.306389808654785, 2.2633931636810303, 2.2260022163391113, 2.1925907135009766, 2.1616415977478027, 2.1320552825927734, 2.10262393951416, 2.072939872741699, 2.0430521965026855, 2.012845277786255, 1.9822957515716553, 1.9516453742980957, 1.9209181070327759, 1.890205979347229, 1.8592673540115356, 1.8273476362228394, 1.7943787574768066, 1.7600386142730713, 1.7243984937667847, 1.6875340938568115, 1.6499934196472168, 1.612219214439392, 1.57450270652771, 1.5369030237197876, 1.4991090297698975, 1.4609405994415283, 1.4223712682724, 1.3833601474761963, 1.3443390130996704, 1.3056485652923584, 1.2678370475769043, 1.2310938835144043, 1.1952532529830933, 1.1601539850234985, 1.1257340908050537, 1.092137098312378, 1.0593922138214111, 1.0275335311889648, 0.9965680241584778, 0.966358482837677, 0.9367967247962952, 0.9078910946846008, 0.879783570766449, 0.8525018095970154, 0.8260131478309631, 0.8001464605331421, 0.7749564051628113, 0.7504231929779053, 0.7266042232513428, 0.703482449054718, 0.6811928749084